In [1]:
!pip install keras-tuner

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
import autokeras as ak
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15927374513689349029
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12738379224506646574
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 18343353596291103167
physical_device_desc: "device: XLA_GPU device"
]


In [7]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [8]:
#Train
#Признаки, связанные с суммой покупки
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Количество в каждой категории
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Сумма в каждой категории
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Минимум в каждой категории
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

#Максимум в каждой категории
max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Средняя сумма в каждой категории
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Соединим все в одну таблицу
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [9]:
train_data = train.head(1000)
test_data = train.tail(200)

In [10]:
import time
start_time = time.time()

In [11]:
model_finder = ak.StructuredDataClassifier()
X_train = train_data.drop('bins', 1)
y_train = train_data['bins']
X_test = test_data.drop('bins', 1)
y_test = test_data['bins']

In [12]:
model_finder.fit(X_train, y_train)

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 2423.6655 - accuracy: 0.218 - ETA: 0s - loss: 2570.0793 - accuracy: 0.291 - ETA: 0s - loss: 1859.9551 - accuracy: 0.299 - ETA: 0s - loss: 1688.7780 - accuracy: 0.296 - ETA: 0s - loss: 1601.8695 - accuracy: 0.296 - 7s 268ms/step - loss: 1564.0187 - accuracy: 0.3029 - val_loss: 495.5297 - val_accuracy: 0.3393
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 924.5522 - accuracy: 0.28 - ETA: 0s - loss: 1244.3237 - accuracy: 0.333 - ETA: 0s - loss: 957.5374 - accuracy: 0.358 - ETA: 0s - loss: 899.9661 - accuracy: 0.34 - ETA: 0s - loss: 862.9788 - accuracy: 0.36 - ETA: 0s - loss: 831.0220 - accuracy: 0.36 - 1s 42ms/step - loss: 820.4752 - accuracy: 0.3690 - val_loss: 393.1793 - val_accuracy: 0.4286
Epoch 3/1000
26/26 [==============================] - ETA: 0s - loss: 426.2672 - accuracy: 0.46 - ETA: 0s - loss: 750.8922 - accuracy: 0.36 - ETA: 0s - loss: 596.7493 - accuracy: 0.38 - ETA: 0s - loss: 543.7

Epoch 22/1000
26/26 [==============================] - ETA: 0s - loss: 10.3642 - accuracy: 0.375 - ETA: 0s - loss: 2.9082 - accuracy: 0.458 - ETA: 0s - loss: 2.2393 - accuracy: 0.45 - ETA: 0s - loss: 2.0382 - accuracy: 0.43 - ETA: 0s - loss: 2.1448 - accuracy: 0.41 - ETA: 0s - loss: 2.2182 - accuracy: 0.41 - 1s 41ms/step - loss: 2.1251 - accuracy: 0.4111 - val_loss: 8.8261 - val_accuracy: 0.3155
Epoch 23/1000
26/26 [==============================] - ETA: 0s - loss: 9.6815 - accuracy: 0.34 - ETA: 0s - loss: 6.5469 - accuracy: 0.42 - ETA: 0s - loss: 4.1554 - accuracy: 0.42 - ETA: 0s - loss: 3.2537 - accuracy: 0.40 - ETA: 0s - loss: 2.8105 - accuracy: 0.40 - 1s 35ms/step - loss: 2.7587 - accuracy: 0.4026 - val_loss: 8.4836 - val_accuracy: 0.3214
Epoch 24/1000
26/26 [==============================] - ETA: 0s - loss: 1.0250 - accuracy: 0.43 - ETA: 0s - loss: 1.1242 - accuracy: 0.45 - ETA: 0s - loss: 1.1612 - accuracy: 0.44 - ETA: 0s - loss: 1.4656 - accuracy: 0.42 - ETA: 0s - loss: 1.5664 -

26/26 [==============================] - ETA: 0s - loss: 1.1346 - accuracy: 0.34 - ETA: 0s - loss: 1.1506 - accuracy: 0.42 - ETA: 0s - loss: 1.1566 - accuracy: 0.42 - ETA: 0s - loss: 1.1684 - accuracy: 0.40 - ETA: 0s - loss: 1.1962 - accuracy: 0.40 - 1s 33ms/step - loss: 1.1945 - accuracy: 0.4026 - val_loss: 6.1504 - val_accuracy: 0.3333
Epoch 45/1000
26/26 [==============================] - ETA: 0s - loss: 4.2561 - accuracy: 0.34 - ETA: 0s - loss: 1.6112 - accuracy: 0.41 - ETA: 0s - loss: 1.4071 - accuracy: 0.41 - ETA: 0s - loss: 1.3372 - accuracy: 0.39 - ETA: 0s - loss: 1.3176 - accuracy: 0.39 - 1s 45ms/step - loss: 1.3116 - accuracy: 0.3966 - val_loss: 6.1953 - val_accuracy: 0.3393


Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 2765.9626 - accuracy: 0.187 - ETA: 0s - loss: 2064.4905 - accuracy: 0.193 - ETA: 0s - loss: 1617.9752 - accuracy: 0.291 - ETA: 0s - loss: 1421.0148 - accuracy: 0.303 - ETA: 0s - loss: 1294.5659 - accuracy: 0.315 - ETA: 0s - loss: 1188.9503 - accuracy: 0.325 - 8s 305ms/step - loss: 1151.3813 - accuracy: 0.3281 - val_loss: 430.7805 - val_accuracy: 0.3274
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 913.1462 - accuracy: 0.34 - ETA: 0s - loss: 741.3643 - accuracy: 0.26 - ETA: 0s - loss: 882.7079 - accuracy: 0.32 - ETA: 0s - loss: 745.6270 - accuracy: 0.34 - ETA: 0s - loss: 657.2378 - accuracy: 0.37 - ETA: 0s - loss: 627.6267 - accuracy: 0.35 - 1s 41ms/step - loss: 605.3034 - accuracy: 0.3642 - val_loss: 290.7763 - val_accuracy: 0.3155
Epoch 3/1000
26/26 [==============================] - ETA: 0s - loss: 331.1411 - accuracy: 0.28 - ETA: 0s - loss: 410.1284 - accuracy: 0.36 - ETA: 0s - loss: 343.01

Epoch 21/1000
26/26 [==============================] - ETA: 0s - loss: 8.6009 - accuracy: 0.53 - ETA: 0s - loss: 10.2082 - accuracy: 0.421 - ETA: 0s - loss: 7.6406 - accuracy: 0.459 - ETA: 0s - loss: 6.5394 - accuracy: 0.42 - ETA: 0s - loss: 6.8212 - accuracy: 0.43 - ETA: 0s - loss: 7.0306 - accuracy: 0.42 - 1s 38ms/step - loss: 6.9342 - accuracy: 0.4363 - val_loss: 34.0731 - val_accuracy: 0.2738
Epoch 22/1000
26/26 [==============================] - ETA: 0s - loss: 9.6953 - accuracy: 0.46 - ETA: 0s - loss: 6.0141 - accuracy: 0.43 - ETA: 0s - loss: 4.4155 - accuracy: 0.48 - ETA: 0s - loss: 4.8410 - accuracy: 0.47 - ETA: 0s - loss: 4.8099 - accuracy: 0.46 - ETA: 0s - loss: 4.3940 - accuracy: 0.48 - 1s 38ms/step - loss: 4.3940 - accuracy: 0.4868 - val_loss: 29.6024 - val_accuracy: 0.3571
Epoch 23/1000
26/26 [==============================] - ETA: 0s - loss: 1.0002 - accuracy: 0.62 - ETA: 0s - loss: 3.2708 - accuracy: 0.48 - ETA: 0s - loss: 3.2349 - accuracy: 0.50 - ETA: 0s - loss: 3.5035

...

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 5312.4497 - accuracy: 0.250 - ETA: 0s - loss: 5598.3774 - accuracy: 0.218 - ETA: 0s - loss: 4402.1016 - accuracy: 0.225 - ETA: 0s - loss: 3642.9326 - accuracy: 0.223 - ETA: 0s - loss: 3222.3574 - accuracy: 0.227 - ETA: 0s - loss: 2928.3777 - accuracy: 0.239 - ETA: 0s - loss: 2656.2168 - accuracy: 0.237 - 8s 316ms/step - loss: 2599.1292 - accuracy: 0.2368 - val_loss: 129.6876 - val_accuracy: 0.2738
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 678.9732 - accuracy: 0.21 - ETA: 0s - loss: 809.7687 - accuracy: 0.27 - ETA: 0s - loss: 934.0013 - accuracy: 0.27 - ETA: 0s - loss: 927.9134 - accuracy: 0.25 - ETA: 0s - loss: 870.2125 - accuracy: 0.25 - ETA: 0s - loss: 854.6909 - accuracy: 0.24 - ETA: 0s - loss: 808.0707 - accuracy: 0.26 - ETA: 0s - loss: 802.4879 - accuracy: 0.27 - ETA: 0s - loss: 780.4071 - accuracy: 0.27 - ETA: 0s - loss: 763.2222 - accuracy: 0.27 - 2s 61ms/step - loss: 733.9521 - acc

Epoch 17/1000
26/26 [==============================] - ETA: 0s - loss: 1.8200 - accuracy: 0.28 - ETA: 0s - loss: 10.0137 - accuracy: 0.306 - ETA: 0s - loss: 10.4794 - accuracy: 0.312 - ETA: 0s - loss: 10.1904 - accuracy: 0.310 - ETA: 0s - loss: 12.6414 - accuracy: 0.286 - ETA: 0s - loss: 15.0301 - accuracy: 0.281 - ETA: 0s - loss: 18.2907 - accuracy: 0.281 - 1s 54ms/step - loss: 17.7304 - accuracy: 0.2788 - val_loss: 1.3845 - val_accuracy: 0.2798
Epoch 18/1000
26/26 [==============================] - ETA: 0s - loss: 13.8814 - accuracy: 0.156 - ETA: 0s - loss: 25.9749 - accuracy: 0.275 - ETA: 0s - loss: 239.5523 - accuracy: 0.27 - ETA: 0s - loss: 174.3128 - accuracy: 0.26 - ETA: 0s - loss: 135.6366 - accuracy: 0.26 - ETA: 0s - loss: 111.8880 - accuracy: 0.25 - ETA: 0s - loss: 95.9458 - accuracy: 0.2625 - 1s 41ms/step - loss: 92.3062 - accuracy: 0.2632 - val_loss: 1.3846 - val_accuracy: 0.2798
Epoch 19/1000
26/26 [==============================] - ETA: 0s - loss: 6.7281 - accuracy: 0.18 

Epoch 1/1000
26/26 [==============================] - ETA: 0s - loss: 926.3316 - accuracy: 0.15 - ETA: 0s - loss: 3472.5625 - accuracy: 0.218 - ETA: 0s - loss: 3744.5803 - accuracy: 0.294 - ETA: 0s - loss: 3110.1792 - accuracy: 0.315 - ETA: 0s - loss: 2789.6064 - accuracy: 0.329 - ETA: 0s - loss: 2559.6511 - accuracy: 0.332 - ETA: 0s - loss: 2343.9568 - accuracy: 0.321 - ETA: 0s - loss: 2133.3557 - accuracy: 0.322 - ETA: 0s - loss: 1972.4603 - accuracy: 0.322 - 10s 402ms/step - loss: 1972.4603 - accuracy: 0.3221 - val_loss: 628.4080 - val_accuracy: 0.3631
Epoch 2/1000
26/26 [==============================] - ETA: 0s - loss: 580.7842 - accuracy: 0.46 - ETA: 0s - loss: 442.0833 - accuracy: 0.45 - ETA: 0s - loss: 863.2164 - accuracy: 0.40 - ETA: 0s - loss: 661.1276 - accuracy: 0.41 - ETA: 0s - loss: 544.3102 - accuracy: 0.41 - ETA: 0s - loss: 500.3031 - accuracy: 0.42 - ETA: 0s - loss: 440.6190 - accuracy: 0.43 - ETA: 0s - loss: 416.1222 - accuracy: 0.44 - ETA: 0s - loss: 389.4311 - accur

26/26 [==============================] - ETA: 0s - loss: 26.1839 - accuracy: 0.812 - ETA: 0s - loss: 17.6359 - accuracy: 0.789 - ETA: 0s - loss: 12.1965 - accuracy: 0.843 - ETA: 0s - loss: 11.4392 - accuracy: 0.853 - ETA: 0s - loss: 11.6886 - accuracy: 0.855 - ETA: 0s - loss: 11.9345 - accuracy: 0.851 - ETA: 0s - loss: 11.2502 - accuracy: 0.856 - ETA: 0s - loss: 11.3273 - accuracy: 0.840 - ETA: 0s - loss: 11.2473 - accuracy: 0.840 - 1s 52ms/step - loss: 11.0673 - accuracy: 0.8438 - val_loss: 168.7473 - val_accuracy: 0.4821
Epoch 16/1000
26/26 [==============================] - ETA: 0s - loss: 11.5702 - accuracy: 0.843 - ETA: 0s - loss: 19.2201 - accuracy: 0.789 - ETA: 0s - loss: 16.7124 - accuracy: 0.786 - ETA: 0s - loss: 17.1412 - accuracy: 0.798 - ETA: 0s - loss: 18.4253 - accuracy: 0.789 - ETA: 0s - loss: 16.2562 - accuracy: 0.796 - ETA: 0s - loss: 15.9864 - accuracy: 0.796 - ETA: 0s - loss: 15.3881 - accuracy: 0.803 - ETA: 0s - loss: 14.6742 - accuracy: 0.806 - ETA: 0s - loss: 14.1

KeyboardInterrupt: 

In [ ]:
model_finder.evaluate(X_test, y_test)

In [ ]:
end_time = time.time()

In [ ]:
print("Time spent", end_time - start_time)